# Imports

In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# Fabrication des train set et test set

In [8]:
Windga_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Windga/WindGa_monthly/Clean/Windga.csv')

In [9]:
#Lecture de l'export mensuel Windga
windga = pd.read_csv(Windga_path, sep=';')

#Visualisation
windga.head()

,Nom centrale,PERIODE,MOIS,ANNEE,Dispo contrat realisee,Dispo technique realisee,Dispo energetique realisee,Production realisee,project_code,date,dispo_consolidated,prod_100p
0,Amelecourt,01/11/2009,11,2009,0.95760,NaN,NaN,3332664.0,AMEL,2009-11-01,0.95760,3.480226e+06
1,Amelecourt,01/10/2016,10,2016,0.97175,0.91491,0.89533,1435415.0,AMEL,2016-10-01,0.89533,1.603225e+06
2,Amelecourt,01/04/2012,4,2012,0.97000,NaN,NaN,2025488.0,AMEL,2012-04-01,0.97000,2.088132e+06
3,Amelecourt,01/03/2019,3,2019,0.98926,0.98820,0.98794,3913817.0,AMEL,2019-03-01,0.98794,3.961594e+06
4,Amelecourt,01/09/2021,9,2021,0.97811,0.96088,0.95953,1019362.0,AMEL,2021-09-01,0.95953,1.062356e+06


In [10]:
#Fabrication aléatoire du WindGa trainset et testset
#2/3 de train pour 1/3 de test
#On ne garde que les mois avec plusieurs valeurs par projet pour stratifier les sets
set_windga = windga[windga.duplicated(subset=['MOIS','project_code'], keep=False)]
windga_train, windga_test = train_test_split(set_windga, test_size=0.33, random_state=42, stratify=set_windga.loc[:,['MOIS','project_code']])

#Visualisation du trainset
windga_train.head()

,Nom centrale,PERIODE,MOIS,ANNEE,Dispo contrat realisee,Dispo technique realisee,Dispo energetique realisee,Production realisee,project_code,date,dispo_consolidated,prod_100p
2187,Montlouby,01/12/2013,12,2013,0.97900,NaN,NaN,2785295.0,MTL1,2013-12-01,0.97900,2.845041e+06
11060,SC - Puech Negre,01/01/2014,1,2014,0.98368,0.80376,0.96124,1812033.0,SAPN,2014-01-01,0.96124,1.885099e+06
6983,Longue Epine,01/09/2019,9,2019,0.95750,0.95446,0.95182,1524427.0,LEPI,2019-09-01,0.95182,1.601592e+06
1000,La Heroudiere,01/06/2021,6,2021,0.95051,0.75378,0.69297,391859.0,HERO,2021-06-01,0.69297,5.654776e+05
10547,Vanault,01/03/2018,3,2018,0.93266,NaN,NaN,1390367.0,VANA,2018-03-01,0.93266,1.490754e+06


In [11]:
#Visualisation du testset
windga_test.head()

,Nom centrale,PERIODE,MOIS,ANNEE,Dispo contrat realisee,Dispo technique realisee,Dispo energetique realisee,Production realisee,project_code,date,dispo_consolidated,prod_100p
12230,Repowering Petit Canal,01/07/2021,7,2021,0.97596,0.71550,0.71040,1574549.0,PCR1,2021-07-01,0.71040,2.216426e+06
4584,4 Bornes - Aumelas,01/12/2016,12,2016,0.92527,0.87059,0.93415,1247950.0,AUQB,2016-12-01,0.93415,1.335920e+06
7191,Lou Paou 12MW,01/06/2018,6,2018,0.89905,0.81636,0.94600,1314583.0,LOPV,2018-06-01,0.94600,1.389623e+06
8942,Plat des Graniers - Corbieres,01/01/2021,1,2021,0.98818,0.98818,0.99403,3063994.0,PLGR,2021-01-01,0.99403,3.082396e+06
6727,Heninel,01/11/2018,11,2018,0.99915,0.99915,NaN,1387786.0,HENI,2018-11-01,0.99915,1.388967e+06


In [12]:
Windga_train_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Train_Test/windga_train.csv')
Windga_test_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Train_Test/windga_test.csv')
    
#Sauvegarde
windga_train.to_csv(Windga_train_path, index=False, sep=';')
windga_test.to_csv(Windga_test_path, index=False, sep=';')

# Dataframe pour le benchmark des rmse

In [13]:
#On ne garde que la production 100% disponible
rmse_train = windga_train.loc[:,['project_code', 'ANNEE','MOIS','prod_100p']]
rmse_train.rename(columns={'project_code':'project','ANNEE':'year','MOIS':'month'}, inplace=True)
#On ajoute la colonne train
rmse_train['train']=1
rmse_train = rmse_train.reindex(columns = ['project', 'year','month','train','prod_100p'])

#Visualisation
rmse_train.sample(5)

,project,year,month,train,prod_100p
9661,SAUV,2011,6,1,3.821331e+06
10147,PDCE,2017,10,1,1.860094e+06
1317,LANE,2012,8,1,9.476116e+05
5064,BARB,2012,7,1,9.562315e+05
8874,PLGR,2013,5,1,3.133375e+06


In [14]:
#On ajoute le testset
rmse_test = windga_test.loc[:,['project_code', 'ANNEE','MOIS','prod_100p']]
rmse_test.rename(columns={'project_code':'project','ANNEE':'year','MOIS':'month'}, inplace=True)
#On ajoute la colonne train
rmse_test['train']=0
rmse_test = rmse_test.reindex(columns = ['project', 'year','month','train','prod_100p'])

#Visualisation
rmse_test.sample(5)

,project,year,month,train,prod_100p
10001,STSI,2011,10,0,1.426995e+06
8803,OUPI,2017,7,0,2.065798e+06
9644,SAUV,2021,7,0,3.507206e+06
1602,LERO,2021,6,0,1.803672e+05
3270,RIBE,2021,3,0,2.205720e+06


In [15]:
#Fusion des deux sets
rmse_windga = pd.concat([rmse_train,rmse_test])

#Indexation par projets et période
rmse_windga.set_index(['project', 'year', 'month'], inplace=True)

In [16]:
#Visualisation
rmse_windga.sample(5)

,,,train,prod_100p
project,year,month,,
FIEN,2011,1,0,3.294539e+06
PLGR,2014,1,1,1.926841e+06
GRPL,2020,7,1,1.357316e+06
TRAY,2017,4,1,2.309000e+03
GAR2,2016,11,0,2.050558e+06


In [17]:
rmse_windga_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Train_Test/rmse_windga.csv')
    
#Sauvegarde
rmse_windga.to_csv(rmse_windga_path, sep=';')